# Токенизация

Токенизация — фундаментальный этап обработки естественного языка, задача которого — разбить текст на осмысленные единицы (токены).

Этими единицами могут быть слова, части слов или даже символы. Исторически использовались простые методы: разделение по пробелам, регулярные выражения для выделения слов и знаков препинания, ручные правила для обработки сокращений. Однако такие подходы плохо масштабировались для языков с агглютинативной морфологией (например, русский или финский) и сложных словосочетаний.

Так же традиционные методы токенизации, такие как разделение по пробелам или ручные правила, часто оказываются неэффективными в реальных сценариях: они плохо справляются с опечатками, редкими словами и многоязычными текстами. Например, слово "веееееерно" или смесь языков в одном предложении могут сломать классический токенайзер.

В современных NLP доминируют алгоритмы субсловной токенизации, такие как BPE (Byte Pair Encoding), которые балансируют между смысловой цельностью токенов и эффективным использованием словаря. В этом ноутбуке мы подробно рассмотрим алгоритм BPE и SentencePiece, а так же научимся работать с токенизаторами библиотеки hugging-face.

Вначале мы импоритруем все библиотеки и функции, которые понадобятся нам в этом ноутбуке.

In [ ]:
import re
import spacy
import numpy as np
import pandas as pd
from datasets import load_dataset

from dataclasses import dataclass
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

from itertools import chain
from typing import List, Dict, Tuple
from collections import Counter, defaultdict

## Загрузка данных

Для демонстрации загрузим параллельный англо-русский корпус [Tatoeba](https://arxiv.org/abs/1812.10464), представленный в работе Artetxe et al. (2019) из библиотеки [Hugging Face Datasets](http://huggingface.co/docs/datasets/loading).

![tatoeba-web.png](attachment:f45503be-7882-4348-b1dd-c9b48d91f4cf.png)

[Tatoeba](https://tatoeba.org/en/sentences/index) - это бесплатная коллекция примеров предложений с переводом, предназначенная для изучающих иностранные языки. Она доступна более чем на 400 языках. Его название происходит от японской фразы «tatoeba» (例えば), означающей «для примера». Он написан и поддерживается сообществом добровольцев по модели открытого сотрудничества. Отдельные авторы известны как татоэбаны.

Мы воспользуемся наборами только для английского и русского языка. Все примеры в этом датасете являются короткими бытовыми фразами: "Let's try something." → "Давайте что-нибудь попробуем!".

Такой формат удобен для обучения трансформеров, которые работают с последовательностями ограниченной длины.

In [ ]:
def load_translation_dataset():
    print("Loading Tatoeba en-ru...")
    try:
        dataset = load_dataset("Helsinki-NLP/tatoeba", lang1="en", lang2="ru", trust_remote_code=True)

    except Exception as e:
        print(f"Error while loading dataset: {e}")
        raise

    print("\nDataset structure:")
    print(dataset)

    print("\nData sample:")
    for i in range(2):
        print(f"EN: {dataset['train'][i]['translation']['en']}")
        print(f"RU: {dataset['train'][i]['translation']['ru']}\n")

    return dataset

In [ ]:
dataset = load_translation_dataset()

## Анализ данных

Давайте быстро взглянем на датасет, чтобы понять с чем мы имеем дело. В этом ноутбуке мы не будем углубляться в методы анализа данных, но все же посмотрим базовую статистику.

Функция analyze_dataset показывает, что средняя длина английских предложений — 7.2 слова, русских — 6.2. Максимальные длины (30 и 28 слов) указывают на наличие выбросов, которые могут требовать обрезки.

Гистограммы демонстрируют правостороннее распределение: большинство предложений короче 15 слов. Эти наблюдения влияют на выбор гиперпараметров модели, например, max_length=64 обеспечивает запас для паддинга, даже если реальные последовательности короче.

In [ ]:
def analyze_dataset(dataset, n_samples: int = 1000):
    samples = dataset['train'].select(range(n_samples))

    en_lengths = [len(s['translation']['en'].split()) for s in samples]
    ru_lengths = [len(s['translation']['ru'].split()) for s in samples]

    print(f"Analysis based on first {n_samples} samples:")
    print(f"\nEnglish sentences:")
    print(f"Average length: {np.mean(en_lengths):.1f} words")
    print(f"Max length: {max(en_lengths)} words")
    print(f"Min length: {min(en_lengths)} words")

    print(f"\nRussian sentences:")
    print(f"Average length: {np.mean(ru_lengths):.1f} words")
    print(f"Max length: {max(ru_lengths)} words")
    print(f"Min length: {min(ru_lengths)} words")

    plt.figure(figsize=(12, 4))

    plt.subplot(1, 2, 1)
    sns.histplot(en_lengths, bins=30)
    plt.title('English Sentence Lengths')
    plt.xlabel('Words')

    plt.subplot(1, 2, 2)
    sns.histplot(ru_lengths, bins=30)
    plt.title('Russian Sentence Lengths')
    plt.xlabel('Words')

    plt.tight_layout()
    plt.show()

    return max(max(en_lengths), max(ru_lengths))

In [ ]:
max_sentence_length = analyze_dataset(dataset)

## Простой токенайзер

Простой токенизатор реализует базовую токенизацию на уровне слов с минимальной предобработкой текста. Класс `BaseTokenizer` имеет несколько ключевых этапов работы:

1. **Препроцессинг текста**: Текст преобразуется в нижний регистр и затем разделяется на токены с использованием регулярного выражения. Это регулярное выражение выделяет слова, содержащие апострофы, а также пунктуацию как отдельные токены. В результате текст разбивается на элементы, такие как слова и знаки препинания, которые затем используются для дальнейшего анализа.

2. **Инициализация словаря**: При инициализации токенизатора задаются специальные токены, такие как `<PAD>`, `<UNK>`, `<BOS>`, `<EOS>`, которые добавляются в словарь. Эти токены имеют фиксированные индексы и используются для обработки данных, например, для выравнивания последовательностей в модели.

3. **Сбор статистики**: Метод для сбора статистики проходит по набору текстов, применяет токенизацию к каждому тексту и подсчитывает частоту появления токенов. Это позволяет создать статистику, на основе которой можно построить словарь для дальнейшего использования в моделях обработки естественного языка.

4. **Особенности подхода**:
   - Преобразование текста в нижний регистр может привести к потере информации о регистре.
   - Токенизация не учитывает морфологические особенности слов, что может привести к проблемам с редкими словами или омонимами.
   - Пунктуация и другие символы выделяются как отдельные токены, что может быть полезно для задач, где важна структура предложений.

Вся работа токенизатора сводится к простым шагам: преобразование текста, токенизация и сбор статистики для построения словаря. Это базовый подход, который можно адаптировать и улучшать для более сложных задач.

Декоратор `@dataclass` автоматически генерирует стандартные методы класса (`__init__`, `__repr__`, `__eq__`)

In [ ]:
@dataclass
class BaseTokenizer:
    language: str
    vocab_size: int
    min_freq: int = 2
    special_tokens: List[str] = None

    def __post_init__(self):
        self.special_tokens = self.special_tokens or ["<PAD>", "<UNK>", "<BOS>", "<EOS>"]
        self.token2id = {token: idx for idx, token in enumerate(self.special_tokens)}
        self.id2token = {idx: token for idx, token in enumerate(self.special_tokens)}

    def preprocess_text(self, text: str) -> List[str]:
        tokens = re.findall(r"\w+[\w']*|['’][a-z]+|[^\w\s]", text.lower())
        return tokens

    def get_stats(self, examples: List[str]) -> Counter:
        counter = Counter()
        for text in examples:
            tokens = self.preprocess_text(text)
            counter.update(tokens)
        return counter

In [ ]:
en_tokenizer = BaseTokenizer(language='en', vocab_size=32000)
ru_tokenizer = BaseTokenizer(language='ru', vocab_size=32000)

Давайте напишем функцию `analyze_token_statistics`, чтобы подсчитать, какие у нас получились уникальные токены и как часто они встречаются.

In [ ]:
def analyze_token_statistics(dataset, tokenizer: BaseTokenizer, n_samples: int = 1000):
    samples = dataset['train'].select(range(n_samples))
    texts = [s['translation'][tokenizer.language] for s in samples]

    stats = tokenizer.get_stats(texts)

    print(f"\nToken statistics for {tokenizer.language}:")
    print(f"Total unique tokens: {len(stats)}")
    print("\nTop 10 most frequent tokens:")
    for token, count in stats.most_common(10):
        print(f"{token}: {count}")

    return stats

In [ ]:
en_stats = analyze_token_statistics(dataset, en_tokenizer)
ru_stats = analyze_token_statistics(dataset, ru_tokenizer)

Разница в количестве токенов для английского (1337) и русского (2065) объясняется особенностями языков: русский имеет более богатую морфологию (окончания, приставки) и больше форм слов. Доминирование пунктуации (. и , в топе) вообще-то указывает на необходимость их предварительной фильтрации или отдельной обработки.

Интересно, что токен " (кавычки) встречается чаще в английском (146 раз) — это может быть связано с особенностями перевода в датасете Tatoeba.

Важно помнить, что такой подход не разбивает слова на субсловные единицы, поэтому редкие слова остаются целыми, увеличивая размер словаря. Для сравнения, BPE-токенизатор и это будет показано в следующих экспериментах

## PBE токенайзер

BPE (Byte Pair Encoding) — это алгоритм для создания подсловных токенов. Он помогает решить проблемы с редкими словами, обеспечивая более компактное представление текста. В отличие от простого токенизатора, который работает на уровне слов, BPE создаёт более мелкие единицы — подслова или даже отдельные символы.

Алгоритм следующий:

1. **Инициализация**: Токенизатор начинается с создания списка символов и специальных токенов, таких как `<PAD>`, `<UNK>`, `<BOS>`, `<EOS>`, которые добавляются в словарь. Начальный словарь включает в себя все символы из текста, а также все уникальные токены.

2. **Обучение**: На основе корпуса текстов токенизатор создаёт статистику для каждого слова, представляя его как последовательность символов (например, буквы, цифры и знаки препинания). Частоты появления этих "символьных слов" подсчитываются, и для каждой пары символов (например, "l" и "o") вычисляется, насколько часто они встречаются рядом. Это важно для выделения наиболее часто встречающихся последовательностей символов.

3. **Слияние пар символов**: На каждом шаге токенизатор ищет наиболее часто встречающуюся пару символов, например, "l" и "o" в слове "hello". Эта пара символов заменяется новым токеном, который объединяет два символа в один. Этот процесс повторяется несколько раз (настраивается параметром `num_merges`), каждый раз добавляя новые токены в словарь и обновляя частотные характеристики текста.

4. **Создание новых токенов**: После каждого слияния обновляется список токенов, а словарь обновляется новыми подсловами. Новый токен становится частью словаря и используется для дальнейшего слияния пар символов. Важно, что новые токены могут быть подсловами или даже словами.

In [ ]:
@dataclass
class BPETokenizer(BaseTokenizer):
    def __post_init__(self):
        super().__post_init__()
        self.merges = {}
        self.vocab = set(self.special_tokens)

    def get_pairs(self, word: List[str]) -> List[Tuple[str, str]]:
        return [(word[i], word[i+1]) for i in range(len(word)-1)]

    def train(self, texts: List[str], num_merges: int):
        word_freqs = defaultdict(int)
        all_chars = set()

        for text in texts:
            tokens = self.preprocess_text(text)
            for token in tokens:
                chars = list(token)
                word_freqs[' '.join(chars)] += 1
                all_chars.update(chars)

        for char in sorted(all_chars):
            if char not in self.token2id:
                idx = len(self.token2id)
                self.token2id[char] = idx
                self.id2token[idx] = char
                self.vocab.add(char)

        word_freqs = defaultdict(int)
        for text in texts:
            tokens = self.preprocess_text(text)
            for token in tokens:
                chars = list(token)
                word = ' '.join(chars)
                word_freqs[word] += 1

        print(f"Training BPE tokenizer for {self.language}...")
        for i in tqdm(range(num_merges)):
            pair_freqs = defaultdict(int)

            for word, freq in word_freqs.items():
                symbols = word.split()
                pairs = self.get_pairs(symbols)
                for pair in pairs:
                    pair_freqs[pair] += freq

            if not pair_freqs:
                break

            best_pair = max(pair_freqs.items(), key=lambda x: x[1])[0]
            new_token = ''.join(best_pair)

            self.merges[best_pair] = new_token
            self.vocab.add(new_token)

            if new_token not in self.token2id:
                idx = len(self.token2id)
                self.token2id[new_token] = idx
                self.id2token[idx] = new_token

            new_word_freqs = defaultdict(int)
            for word, freq in word_freqs.items():
                symbols = word.split()
                i = 0
                new_symbols = []
                while i < len(symbols):
                    if i < len(symbols)-1 and (symbols[i], symbols[i+1]) == best_pair:
                        new_symbols.append(new_token)
                        i += 2
                    else:
                        new_symbols.append(symbols[i])
                        i += 1
                new_word = ' '.join(new_symbols)
                new_word_freqs[new_word] += freq

            word_freqs = new_word_freqs

            if (i + 1) % 1000 == 0:
                print(f"Merges completed: {i+1}/{num_merges}")
                print(f"Current vocabulary size: {len(self.token2id)}")

    def tokenize(self, text: str) -> List[int]:
        tokens = self.preprocess_text(text)
        result = [self.token2id['<BOS>']]

        for token in tokens:
            symbols = list(token)

            while len(symbols) > 1:
                pairs = self.get_pairs(symbols)
                pair_to_merge = None
                for pair in pairs:
                    if pair in self.merges:
                        pair_to_merge = pair
                        break
                if not pair_to_merge:
                    break

                i = 0
                new_symbols = []
                while i < len(symbols):
                    if i < len(symbols)-1 and (symbols[i], symbols[i+1]) == pair_to_merge:
                        new_symbols.append(self.merges[pair_to_merge])
                        i += 2
                    else:
                        new_symbols.append(symbols[i])
                        i += 1
                symbols = new_symbols

            for symbol in symbols:
                if symbol in self.token2id:
                    result.append(self.token2id[symbol])
                else:
                    result.append(self.token2id['<UNK>'])

        result.append(self.token2id['<EOS>'])
        return result

Когда необходимо применить токенизатор для преобразования текста в последовательность токенов, алгоритм сначала разбивает текст на базовые символы, затем повторно сливает пары символов, используя уже построенный словарь слияний. Каждое слово в тексте представляется как последовательность подслов (или токенов), которые были созданы на этапе обучения.

Количество слияний (параметр `num_merges`) определяет, сколько раз алгоритм будет объединять символы в новые токены. Чем больше количество слияний, тем более крупные и информативные токены могут быть созданы. Однако важно, что слишком большое количество слияний может привести к потере мелких деталей в тексте.

Этот алгоритм хорошо работает с большими текстовыми корпусами и помогает моделям лучше справляться с редкими или незнакомыми словами, заменяя их подсловами из более частых комбинаций символов. Кроме того, BPE поддерживает работу с любыми языками, даже если они используют необычные или сложные алфавиты, так как начинает с базовых символов.

In [ ]:
en_bpe = BPETokenizer(language='en', vocab_size=32000)
ru_bpe = BPETokenizer(language='ru', vocab_size=32000)

n_samples = 80000
train_samples = dataset['train'].select(range(n_samples))
en_texts = [s['translation']['en'] for s in train_samples]
ru_texts = [s['translation']['ru'] for s in train_samples]

en_bpe.train(en_texts, num_merges=3000)
ru_bpe.train(ru_texts, num_merges=3000)

print(f"English vocabulary size: {len(en_bpe.token2id)}")
print(f"Russian vocabulary size: {len(ru_bpe.token2id)}")

In [ ]:
def test_tokenization(text: str, tokenizer: BPETokenizer):
    print(f"\nOriginal text: {text}")

    token_ids = tokenizer.tokenize(text)
    print(f"Token IDs: {token_ids}")

    tokens = [tokenizer.id2token[id] for id in token_ids]
    print(f"Tokens: {tokens}")

    return token_ids

In [ ]:
en_sample = dataset['train'][0]['translation']['en']
ru_sample = dataset['train'][0]['translation']['ru']

print("English tokenization:")
en_tokens = test_tokenization(en_sample, en_bpe)

print("\nRussian tokenization:")
ru_tokens = test_tokenization(ru_sample, ru_bpe)

В общем, BPE эффективно решает проблему редких и сложных слов, улучшая качество токенизации и производительность NLP-моделей.

Однако даже после обучения заметны артефакты. Например, слово "useless" разбивается на ["us", "el", "ess"], а "бесполезно" — на ["бес", "пол", "ез", "но"]. Это следствие ограниченного числа слияний и отсутствия явного учета морфемных границ в нашей учебной реализации.

В готовых токенизаторах (например, от Hugging Face) такие проблемы смягчаются за счет предобучения на огромных корпусах и десятков тысяч слияний.

## Подготовка батчей

Функция prepare_batch преобразует токенизированные последовательности в тензоры, пригодные для обучения.

Каждое предложение дополняется до фиксированной длины (max_length=64) специальным токеном <PAD>, а маски внимания указывают модели игнорировать эти "пустые" позиции.

Например, предложение из 24 токенов превращается в вектор длины 64, где 40 последних элементов — нули (ID <PAD>). Маскирование критично для трансформеров, так как механизм внимания иначе будет учитывать бессмысленные паддинг-токены, искажая веса.

In [ ]:
def prepare_batch(batch: List[Dict],
                 src_tokenizer: BPETokenizer,
                 tgt_tokenizer: BPETokenizer,
                 max_length: int):

    src_texts = [item['translation']['en'] for item in batch]
    tgt_texts = [item['translation']['ru'] for item in batch]

    src_tokens = [src_tokenizer.tokenize(text) for text in src_texts]
    tgt_tokens = [tgt_tokenizer.tokenize(text) for text in tgt_texts]

    src_padded = []
    tgt_padded = []
    src_masks = []
    tgt_masks = []

    for src, tgt in zip(src_tokens, tgt_tokens):
        if len(src) > max_length:
            src_pad = src[:max_length]
            src_mask = [1] * max_length
        else:
            src_pad = src + [src_tokenizer.token2id['<PAD>']] * (max_length - len(src))
            src_mask = [1] * len(src) + [0] * (max_length - len(src))

        if len(tgt) > max_length:
            tgt_pad = tgt[:max_length]
            tgt_mask = [1] * max_length
        else:
            tgt_pad = tgt + [tgt_tokenizer.token2id['<PAD>']] * (max_length - len(tgt))
            tgt_mask = [1] * len(tgt) + [0] * (max_length - len(tgt))

        src_padded.append(src_pad)
        tgt_padded.append(tgt_pad)
        src_masks.append(src_mask)
        tgt_masks.append(tgt_mask)

    return {
        'src_tokens': np.array(src_padded),
        'tgt_tokens': np.array(tgt_padded),
        'src_mask': np.array(src_masks),
        'tgt_mask': np.array(tgt_masks)
    }


In [ ]:
test_samples = dataset['train'].select(range(5))
prepared_data = prepare_batch(test_samples, en_bpe, ru_bpe, max_length=64)

print("Prepared batch shapes:")
for key, value in prepared_data.items():
    print(f"{key}: {value.shape}")

print("\nExample source tokens:")
print(prepared_data['src_tokens'][0])
print("\nCorresponding mask:")
print(prepared_data['src_mask'][0])

In [ ]:
def verify_bpe_tokenization(tokenizer: BPETokenizer, text: str):
    print(f"Original text: {text}")

    base_tokens = tokenizer.preprocess_text(text)
    print(f"\nBase tokenization: {base_tokens}")

    print(f"\nNumber of merges learned: {len(tokenizer.merges)}")
    print("Sample merges (first 5):")
    for pair, merged in list(tokenizer.merges.items())[:5]:
        print(f"{pair} -> {merged}")

    print(f"\nVocabulary size: {len(tokenizer.token2id)}")
    print("Sample vocabulary items (first 10):")
    for token, idx in list(tokenizer.token2id.items())[:10]:
        print(f"{token}: {idx}")

    tokens = tokenizer.tokenize(text)
    decoded = [tokenizer.id2token[id] for id in tokens]

    print(f"\nFinal tokenization:")
    print(f"Token IDs: {tokens}")
    print(f"Decoded tokens: {decoded}")

print("Testing English tokenizer:")
verify_bpe_tokenization(en_bpe, dataset['train'][0]['translation']['en'])

## Hugging Face токенизаторы

Использование готового токенизатора через AutoTokenizer демонстрирует преимущества стандартизированных инструментов.

Модель opus-mt-en-ru использует предобученный BPE-словарь, оптимизированный для пары языков. Токенизатор автоматически добавляет служебные токены, обрабатывает регистр и редкие символы.

При обработке датасета функция map применяет токенизацию параллельно ко всем примерам, что ускоряет работу за счет батчинга.

In [ ]:
from transformers import AutoTokenizer

def prepare_data_with_hf(
    dataset,
    model_name: str = "Helsinki-NLP/opus-mt-en-ru",
    max_length: int = 128,
    batch_size: int = 32
):

    tokenizer = AutoTokenizer.from_pretrained(model_name)

    def preprocess_function(examples):
        source_texts = [item['en'] for item in examples['translation']]
        target_texts = [item['ru'] for item in examples['translation']]

        source_encoding = tokenizer(
            source_texts,
            padding='max_length',
            truncation=True,
            max_length=max_length,
            return_tensors='np'
        )

        target_encoding = tokenizer(
            target_texts,
            padding='max_length',
            truncation=True,
            max_length=max_length,
            return_tensors='np'
        )

        return {
            'input_ids': source_encoding['input_ids'],
            'attention_mask': source_encoding['attention_mask'],
            'labels': target_encoding['input_ids'],
            'decoder_attention_mask': target_encoding['attention_mask']
        }

    processed_dataset = dataset['train'].map(
        preprocess_function,
        batched=True,
        batch_size=batch_size,
        remove_columns=dataset['train'].column_names
    )

    return processed_dataset, tokenizer

In [ ]:
processed_data, hf_tokenizer = prepare_data_with_hf(dataset)

## Test

In [ ]:
def print_custom_bpe_data_shape(prepared_data):
    print("\n" + "="*50)
    print("Custom BPE Tokenizer Data Structure:")
    print("Shape of prepared batches:")
    for key, array in prepared_data.items():
        print(f"{key}: {array.shape} (dtype: {array.dtype})")

    print("\nSample data from first batch:")
    print("Source tokens (first example):")
    print(prepared_data['src_tokens'][0])
    print("\nTarget tokens (first example):")
    print(prepared_data['tgt_tokens'][0])
    print("\nSource mask (first example):")
    print(prepared_data['src_mask'][0])
    print("="*50 + "\n")

def print_hf_data_details(processed_dataset, tokenizer):
    print("\n" + "="*50)
    print("Hugging Face Tokenizer Data Structure:")
    print(f"Dataset features: {processed_dataset.features}")
    print(f"Number of examples: {len(processed_dataset)}")

    first_example = processed_dataset[0]
    print("\nFirst example details:")
    print("Input IDs shape:", len(first_example['input_ids']))
    print("Decoded input:", tokenizer.decode(first_example['input_ids'], skip_special_tokens=True))
    print("Labels shape:", len(first_example['labels']))
    print("Decoded labels:", tokenizer.decode(first_example['labels'], skip_special_tokens=True))
    print("Attention mask sample:", first_example['attention_mask'][:10])
    print("="*50 + "\n")

In [ ]:
test_samples = dataset['train'].select(range(5))
prepared_data = prepare_batch(test_samples, en_bpe, ru_bpe, max_length=64)
print_custom_bpe_data_shape(prepared_data)


processed_data, hf_tokenizer = prepare_data_with_hf(dataset)
print_hf_data_details(processed_data, hf_tokenizer)